# OBJ1 数据准备 + OBJ2 Baseline（TF‑IDF + Logistic Regression）

本 notebook 面向 Sentiment140（training.1600000.processed.noemoticon.csv），完成：

- **OBJ1**：清洗/预处理 + **60/10/30**（train/val/test）分层划分
- **OBJ2**：建立 **baseline supervised sentiment classifier（TF‑IDF + Logistic Regression）**

> 备注：保留否定词、感叹号/问号、重复字母、hashtag 等情感信号；避免过度清洗导致信息损失。

In [ ]:
# !pip -q install scikit-learn pandas numpy joblib


## 1. 读取数据（Sentiment140）

原始字段：`target, ids, date, flag, user, text`。标准 Sentiment140 通常只有 **0(neg)** 与 **4(pos)**。

In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path("Dataset/training.1600000.processed.noemoticon.csv")

COLS = ["target","ids","date","flag","user","text"]

df = pd.read_csv(
    DATA_PATH,
    encoding="ISO-8859-1",
    header=None,
    names=COLS,
    usecols=["target","text"],  # baseline 只用标签与文本
    dtype={"target":"int32","text":"string"},
)

# 只保留 0/4（二分类）
df = df[df["target"].isin([0,4])].copy()
df["label"] = (df["target"] == 4).astype("int8")  # 1=pos, 0=neg

print(df.shape)
df.head()


## 2. 轻量但“情感友好”的预处理

原则：
- ✅ 保留否定信息：**不删除 stopwords**（或至少保留 not/no/never/n't）
- ✅ 保留强度信号：`! ?`、重复字符（soooo）、全部大写的强调
- ✅ 保留 hashtag 语义（#happy）
- ✅ 只移除低价值噪声：URL、@提及、HTML、冗余空白

> 这里采用“最小损失清洗”，把复杂特征交给 TF‑IDF（含 bigram）学习。

In [ ]:
import re

URL_RE = re.compile(r"(https?://\S+|www\.\S+)")
USER_RE = re.compile(r"@\w+")
HTML_RE = re.compile(r"&\w+;")
# 仅移除除常见情感符号外的杂字符：保留 ! ? # '（用于n't）
BAD_CHARS_RE = re.compile(r"[^0-9A-Za-z\s!?#!'’]")

MULTISPACE_RE = re.compile(r"\s+")
REPEAT_RE = re.compile(r"(.)\1{3,}")  # 超过3次重复归一

def normalize_repeats(text: str) -> str:
    # soooo -> soo（保留一定强度信息）
    return REPEAT_RE.sub(r"\1\1", text)

def preprocess(text: str) -> str:
    if text is None:
        return ""
    t = str(text)
    t = HTML_RE.sub(" ", t)
    t = URL_RE.sub(" <URL> ", t)
    t = USER_RE.sub(" <USER> ", t)
    t = normalize_repeats(t)
    # 保留 hashtag：#happy -> HASHTAG_happy（让 tfidf 能吃到）
    t = re.sub(r"#(\w+)", r" HASHTAG_\1 ", t)
    # 统一引号，保留 n't
    t = t.replace("’", "'")
    t = BAD_CHARS_RE.sub(" ", t)
    t = MULTISPACE_RE.sub(" ", t).strip()
    return t.lower()

df["text_clean"] = df["text"].map(preprocess)

df[["text","text_clean","label"]].head(10)


## 3. 分层划分：60/10/30（train/val/test）

按照 Interim Report 的 OBJ1 要求进行固定比例划分，并使用 `stratify` 保持正负样本比例一致。

In [ ]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

# 先切出 train 60% 与 temp 40%
train_df, temp_df = train_test_split(
    df[["text_clean","label"]],
    test_size=0.40,
    random_state=RANDOM_STATE,
    stratify=df["label"],
)

# 再把 temp 40% 切成 val 10% 与 test 30%
# val 在总量占 10% => 在 temp 中占 10/40 = 0.25
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.75,
    random_state=RANDOM_STATE,
    stratify=temp_df["label"],
)

def dist(name, d):
    return name, len(d), float(d["label"].mean())

for item in [dist("train", train_df), dist("val", val_df), dist("test", test_df)]:
    print(item)


## 4. OBJ2 Baseline：TF‑IDF + Logistic Regression

推荐设置：
- `ngram_range=(1,2)`：捕捉 **not good** 这类否定 bigram
- `sublinear_tf=True`：减少超高频词的影响
- `min_df`：过滤极低频噪声
- `class_weight='balanced'`：以防比例不完全平衡

> 注意：baseline 不做 embedding、不用深度网络，符合 OBJ2 要求。

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf_lr = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1,2),
        min_df=2,
        max_df=0.95,
        sublinear_tf=True,
    )),
    ("clf", LogisticRegression(
        max_iter=200,
        solver="saga",
        n_jobs=-1,
        class_weight="balanced",
        random_state=RANDOM_STATE,
    ))
])

X_train, y_train = train_df["text_clean"], train_df["label"]
X_val, y_val     = val_df["text_clean"],   val_df["label"]
X_test, y_test   = test_df["text_clean"],  test_df["label"]

tfidf_lr.fit(X_train, y_train)

print("val_acc:", tfidf_lr.score(X_val, y_val))
print("test_acc:", tfidf_lr.score(X_test, y_test))


## 5. 评估：Accuracy + Precision/Recall/F1 + Confusion Matrix

OBJ4 会更系统，但这里先给 baseline 的核心评估结果。

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = tfidf_lr.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


## 6. 保存产物（给 GUI/后续对比复用）

保存：
- `baseline_tfidf_lr.joblib`：整个 pipeline（含向量化器+模型）
- `splits/*.csv`：三份划分（便于复现实验）

In [ ]:
import joblib
from pathlib import Path

OUT_DIR = Path("artifacts")
OUT_DIR.mkdir(exist_ok=True)

joblib.dump(tfidf_lr, OUT_DIR / "baseline_tfidf_lr.joblib")

SPLIT_DIR = OUT_DIR / "splits"
SPLIT_DIR.mkdir(exist_ok=True)

train_df.to_csv(SPLIT_DIR / "train_60.csv", index=False)
val_df.to_csv(SPLIT_DIR / "val_10.csv", index=False)
test_df.to_csv(SPLIT_DIR / "test_30.csv", index=False)

print("Saved to:", OUT_DIR.resolve())


## 7. 快速推理：输出用于 Happiness Index 的分数

Logistic Regression 支持 `predict_proba`，可以把 `P(pos)` 当作情绪强度，再做窗口聚合（daily/weekly）。

In [ ]:
import numpy as np

def predict_sentiment(texts):
    texts_clean = [preprocess(t) for t in texts]
    proba_pos = tfidf_lr.predict_proba(texts_clean)[:,1]
    label = (proba_pos >= 0.5).astype(int)
    return proba_pos, label

samples = [
    "I am sooooo happy!!!",
    "not good at all...",
    "This is fine? maybe."
]

proba, label = predict_sentiment(samples)
list(zip(samples, proba.round(4), label))
